# A Production-Ready Workflow with Flotorch ADK

This notebook demonstrates a comprehensive, production-ready implementation that combines all advanced features of the Flotorch ADK. We will build a sophisticated agent that leverages a hybrid model:

- **Remote Configuration**: Its core persona is loaded from the Flotorch UI using `FlotorchADKAgent`.
- **Local Augmentation**: It is enhanced at runtime with locally-defined custom tools.
- **Full Memory Stack**: It is equipped with both short-term session memory and long-term persistent knowledge.

### Prerequesit
Configure agent, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

## 1. Environment Setup and Imports

The following cells install dependencies, define API credentials, and import all necessary components for handling a complete workflow, including agent clients, memory services, session managers, and tools.

In [ ]:
# install flotorch adk package
%pip install --pre flotorch[adk]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
AGENT_NAME = "<flotorch agent name>"
MEMORY_PROVIDER = "<flotorch memory provider>"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

In [ ]:
# Import necessary libraries
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.memory import FlotorchMemoryService
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types
from google.adk.tools import FunctionTool

print("Imported necessary libraries successfully")

## 2. Defining Local Tools and Memory Services

Before initializing the agent, we prepare its local components. This includes defining a Python function for a custom tool (`comprehensive_text_analyzer`) and initializing the services that will handle the agent's long-term (`FlotorchMemoryService`) and short-term (`FlotorchADKSession`) memory.

In [ ]:
def comprehensive_text_analyzer(text: str) -> str:
    """Comprehensive text analysis with detailed statistics."""
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    paragraphs = text.count('\n\n') + 1
    avg_word_length = sum(len(word) for word in text.split()) / word_count if word_count > 0 else 0
    
    return f"""Comprehensive Text Analysis:
                - Word Count: {word_count}
                - Character Count (with spaces): {char_count}
                - Character Count (without spaces): {char_count_no_spaces}
                - Sentence Count: {sentences}
                - Paragraph Count: {paragraphs}
                - Average Word Length: {avg_word_length:.2f} characters"""

custom_tools = [FunctionTool(func=comprehensive_text_analyzer)]

print("Advanced custom tools is created")

In [ ]:
# Initialize Memory
memory_service = FlotorchMemoryService(
    name=MEMORY_PROVIDER, 
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

# Initialize Session
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY, base_url=FLOTORCH_BASE_URL
)

print("Initialized Memory and Session")

## 3. Initializing the Agent with Local Augmentations

Here, we initialize the `FlotorchADKAgent`. The client fetches the agent's base configuration from the platform, and we enhance it at runtime by passing two key arguments:

- `custom_tools`: Provides the agent with our locally defined text analyzer tool.
- `enable_memory=True`: Activates the agent's built-in ability to use the long-term memory service.

In [ ]:
flotorch_client = FlotorchADKAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    enable_memory=True,
    custom_tools=custom_tools
)

agent = flotorch_client.get_agent()
print(f"Advanced FlotorchADKAgent '{agent.name}' created.")

## 4. Assembling and Running the Complete Workflow

Finally, we set up the `Runner`, providing it with our fully-augmented agent and both the session and memory services. The interactive chat below will demonstrate the agent's ability to seamlessly use its custom tools and recall information from past sessions, showcasing the power of this integrated architecture.

In [ ]:
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service
)

async def chat_with_agent(query,session_id):
    """
    Chat function that returns the agent's response and saves the interaction to memory.
    """
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=session_id, new_message=content)
    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                # Save the session to memory after getting a response
                completed_session = await runner.session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=session_id)
                await memory_service.add_session_to_memory(completed_session)
                return event.content.parts[0].text
    return "Sorry, I couldn't process that request."

print("Comprehensive runner configured successfully.")

In [ ]:
# === Session 1: Teach the agent a fact and use a tool ===
session1 = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID)
print(f"Started Session 1 with ID: {session1.id}\n")

response1 = await chat_with_agent("My name is Vijay and I am a software engineer", session1.id)
print("user: My name is Vijay and I am a software engineer.")
print(f"Response: {response1}")

response2 = await chat_with_agent("Analyze this text: 'The weather is sunny today outside.'", session1.id)
print(f"\nQuery: Analyze this text: 'The weather is sunny today outside.'")
print(f"Response: {response2}")

print("--- Session 1 ended and memory saved ---")

In [ ]:
# === Session 2: Start a new conversation and recall the fact ===
session2 = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID)
print(f"\nStarted Session 2 with ID: {session2.id}\n")

response2 = await chat_with_agent("What is my name?", session2.id)
print(f"Query: What is my name?")
print(f"Response: {response2}")

## Summary

This notebook provided a template for a comprehensive, production-ready AI agent. By combining remote configuration management with the flexibility of local custom tools and a powerful dual-memory system, this architecture allows for the creation of scalable, maintainable, and highly capable AI applications.

### Key Achievements:

- **Hybrid Agent Model**: We successfully loaded a base agent from the Flotorch UI using `FlotorchADKAgent` and dynamically augmented it with locally-defined tools, combining the benefits of centralized management and local flexibility.
- **Integrated Intelligence**: The agent seamlessly handled tasks requiring either its innate knowledge, its custom tools, or its long-term memory, demonstrating a comprehensive problem-solving ability.
- **Scalable Architecture**: This pattern of separating agent configuration from the core application logic while allowing for runtime augmentation is highly scalable and maintainable, making it ideal for real-world deployment.